In [1]:
%load_ext autoreload
%autoreload 2
import widget_plot

### Run the following cell to start with an Earth-like atmosphere

Description of each parameter can be viewed as mouseovers. Press enter or click elsewhere after making changes to parameters.

In [2]:
widget_plot.plotFinal()

## System of equations

### Thermal fluxes at RC boundary

Equation (13) in Robinson & Catling (2012) reads: $$F^+(\tau)=\sigma T_0^4e^{D\tau}\left[e^{-D\tau_0}+\frac{1}{(D\tau_0)^{4\beta/n}}\left(\Gamma\left(1+\frac{4\beta}{n},D\tau\right)-\Gamma\left(1+\frac{4\beta}{n},D\tau_0\right)\right)\right],$$ where the incomplete Gamma function as defined in Appendix is:$$\Gamma(a,x)\equiv \int_x^{\infty}t^{a-1}e^{-t}dt,$$ whereas scipy.special.gammaincc(a,x) computes:$$Q(a,x)=\frac{1}{\Gamma (a)}\int_x^{\infty}t^{a-1}e^{-t}dt\Rightarrow \Gamma(a,x)=Q(a,x)\Gamma (a);$$

which, at $\tau=\tau_\mathrm{rc}$, must agree with (equation 19):$$F^+(\tau)=\frac{F_1}{2}\left[1+\frac{D}{k_1}+\left(1-\frac{D}{k_1}\right)e^{-k_1\tau}\right]+\frac{F_2}{2}\left[1+\frac{D}{k_2}+\left(1-\frac{D}{k_2}\right)e^{-k_2\tau}\right]+\frac{F_i}{2}(2+D\tau).$$

So the first equality is:
\begin{equation}\sigma T_0^4e^{D\tau_\mathrm{rc}}\left[e^{-D\tau_0}+\frac{1}{(D\tau_0)^{4\beta/n}}\left(\Gamma\left(1+\frac{4\beta}{n},D\tau_\mathrm{rc}\right)-\Gamma\left(1+\frac{4\beta}{n},D\tau_0\right)\right)\right]=\frac{F_1}{2}\left[1+\frac{D}{k_1}+\left(1-\frac{D}{k_1}\right)e^{-k_1\tau_\mathrm{rc}}\right]+\frac{F_2}{2}\left[1+\frac{D}{k_2}+\left(1-\frac{D}{k_2}\right)e^{-k_2\tau_\mathrm{rc}}\right]+\frac{F_i}{2}(2+D\tau_\mathrm{rc})
\end{equation}

And the second one (equation 21):$$\sigma T_0^4\left(\frac{\tau_\mathrm{rc}}{\tau_0}\right)^{4\beta/n}=\frac{F_1}{2}\left[1+\frac{D}{k_1}+\left(\frac{k_1}{D}-\frac{D}{k_1}\right)e^{-k_1\tau_\mathrm{rc}}\right]+\frac{F_2}{2}\left[1+\frac{D}{k_2}+\left(\frac{k_2}{D}-\frac{D}{k_2}\right)e^{-k_2\tau_\mathrm{rc}}\right]+\frac{F_i}{2}(2+D\tau_\mathrm{rc})$$

with relevant parameters: $T_0$, $\tau_\mathrm{rc}$, $\tau_0$, $\alpha$, $\gamma$, $n$, $F_1$, $k_1$, $F_2$, $k_2$, $F_i$; and with the relationship $$\tau=\tau_0\left(\frac{p}{p_0}\right)^n.$$

($\tau_0$ and $\tau_\mathrm{rc}$ should be solved and then entered as arguments into f)